# Combining LCEL Chains

## Setup

#### After you download the code from the github repository in your computer
In terminal:
* cd project_name
* pyenv local 3.11.4
* poetry install
* poetry shell

#### To open the notebook with Jupyter Notebooks
In terminal:
* jupyter lab

Go to the folder of notebooks and open the right notebook.

#### To see the code in Virtual Studio Code or your editor of choice.
* open Virtual Studio Code or your editor of choice.
* open the project-folder
* open the 007-main-ops-lcel-chain.py file

## Create your .env file
* In the github repo we have included a file named .env.example
* Rename that file to .env file and here is where you will add your confidential api keys. Remember to include:
* OPENAI_API_KEY=your_openai_api_key
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=your_langchain_api_key
* LANGCHAIN_PROJECT=your_project_name

We will call our LangSmith project **007-main-ops-lcel-chain**.

## Connect with the .env file located in the same directory of this notebook

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [ ]:
#!pip install python-dotenv

In [5]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [ ]:
#!pip install langchain

In [3]:
#!pip install langchain lanchain-community

## Connect with an LLM and start a conversation with it

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [5]:
#!pip install langchain-openai

* For this project, we will use OpenAI's gpt-3.5-turbo

In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

## Coercion: a chain inside another chain
* Remember: almost any component in LangChain (prompts, models, output parsers, etc) can be used as a Runnable.
* **Runnables can be chained together using the pipe operator `|`. The resulting chains of runnables are also runnables themselves**.

In [18]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a sentence about {politician}")

model = ChatOpenAI(model="gpt-3.5-turbo")

chain = prompt | model | StrOutputParser()

In [19]:
chain.invoke("Chamberlain")

'Chamberlain was a British politician who is best known for his policy of appeasement towards Nazi Germany in the years leading up to World War II.'

#### Coercion: combine a chain with other Runnables to create a new chain.
* See how in the `composed_chain` we are including the previous `chain`:

In [20]:
from langchain_core.output_parsers import StrOutputParser

historian_prompt = ChatPromptTemplate.from_template("Was {politician} positive for Humanity?")

composed_chain = {"politician": chain} | historian_prompt | model | StrOutputParser()

In [21]:
composed_chain.invoke({"politician": "Lincoln"})

"Yes, Abraham Lincoln is considered to have had a positive impact on humanity. His leadership during the Civil War helped to preserve the unity of the United States and ultimately led to the end of slavery. The Emancipation Proclamation was a significant step towards equality and justice for all individuals, and Lincoln's efforts to abolish slavery have had a lasting impact on society. Additionally, Lincoln's leadership and dedication to upholding the principles of democracy and freedom have inspired generations of Americans and individuals around the world."

In [22]:
composed_chain.invoke({"politician": "Attila"})

"Attila the Hun's conquests and actions were not necessarily positive for humanity. He was known for his brutal tactics, including pillaging and destroying cities, and his reign brought suffering and devastation to many people. While he was certainly a powerful and formidable leader, his legacy is more often associated with violence and destruction rather than positive contributions to humanity."

## Another example: a chain inside another chain

In [25]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt1 = ChatPromptTemplate.from_template("what is the country {politician} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what continent is the country {country} in? respond in {language}"
)

model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"country": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"politician": "Miterrand", "language": "French"})

"Le continent où se trouve la France, dont François Mitterrand était originaire, est l'Europe."

## Fallback for Chains
* When working with language models, you may often encounter issues from the underlying APIs, whether these be rate limiting or downtime. Therefore, as you go to move your LLM applications into production it becomes more and more important to safeguard against these. That's why LangChain introduced the concept of fallbacks.
* A fallback is an alternative plan that may be used in an emergency.
* Fallbacks can be applied not only on the LLM level but on the whole runnable level. This is important because often times different models require different prompts. So if your call to OpenAI fails, you don't just want to send the same prompt to Anthropic - you probably want to use a different prompt template and send a different version there.
* We can create fallbacks for LCEL chains. Here we do that with two different models: ChatOpenAI (with a bad model name to easily create a chain that will error) and then normal OpenAI (which does not use a chat model). Because OpenAI is NOT a chat model, you likely want a different prompt.

In [6]:
# First let's create a chain with a ChatModel
# We add in a string output parser here so the outputs between the two are the same type
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a funny assistant who always includes a joke in your response",
        ),
        ("human", "Who is the best {sport} player worldwide?"),
    ]
)
# Here we're going to use a bad model name to easily create a chain that will error
chat_model = ChatOpenAI(model="gpt-fake")

bad_chain = chat_prompt | chat_model | StrOutputParser()

In [7]:
# Now lets create a chain with the normal OpenAI model
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

prompt_template = """Instructions: You're a funny assistant who always includes a joke in your response.

Question: Who is the best {sport} player worldwide?"""

prompt = PromptTemplate.from_template(prompt_template)

llm = OpenAI()

good_chain = prompt | llm

In [8]:
# We can now create a final chain which combines the two
chain = bad_chain.with_fallbacks([good_chain])

chain.invoke({"sport": "soccer"})

"\n\nResponse: Well, it depends on who you ask. Some might say Messi, others might say Ronaldo. But I personally think it's my grandma, she can kick a ball farther than anyone I know!"

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 007-main-ops-lcel-chain.py
* In terminal, make sure you are in the directory of the file and run:
    * python 007-main-ops-lcel-chain.py